In [1]:

import numpy as np
import pandas as pd
import sklearn
import scipy
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
#from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import preprocessing
#data = pd.read_csv(r'/home/supriya/CDS_vectorization_v2x.csv',dtype={'Error_Label':object} )
data = pd.read_csv(r'/home/supriya/Hybrid_v2_v2x.csv',dtype={'Error_Label':object} )
print(data.shape)

(67677, 1401)


In [2]:
# split labels and return new dataframe
#Create independent and Dependent Features
ERROR_LABEL=6
def split_label_dict(df):
    # Change 'label' to whatever naming used in the original DataFrame.
    temp_df = df['Error_Label'].str.split("", n = -1, expand = True)
    for i in range(1, 19):
        df[f'l{i}'] = temp_df[i]
        df[f'l{i}'] = pd.to_numeric(df[f'l{i}'])
    return df

newdata=split_label_dict(data)
#print(newdata)
#Create independent and Dependent Features
df = pd.DataFrame(data)
new_df = df.drop(['Error_Label','l18','l17','l16','l15','l14','l13','l12','l11','l10','l9','l8','l7','l6','l5','l3','l4','l2','l1', 'ID'],axis=1)
#new_df = data.drop(['Error_Label','ID'],axis=1)
X = new_df
# Store the variable we are predicting 
Y = newdata[f"l{ERROR_LABEL}"]

# Print the shapes of X & Y
print(X.shape,Y.shape)


(67677, 1399) (67677,)


In [3]:

from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=69,stratify=Y)


In [4]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [5]:
## Hyper Parameter Optimization

params={
    "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15, 20,25,30,35],
 "min_child_weight" : [ 1, 3, 5, 7,9,11,15,19,24,29,30 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
    "booster":['gbtree', 'gblinear','dart'],
    "eta":[0.05, 0.10, 0.15, 0.20, 0.25, 0.30,.40,.50,.60,.70,.80,.90],
    "tree_method":['auto', 'exact', 'approx', 'hist', 'gpu_hist'],
"grow_policy":['depthwise', 'lossguide'],
    "reg_lambda":[0,1,2,3,4],
    "alpha":[0,1,2,3,4],
    "n_estimators":[100,200,300,400,500],
    "sampling_method":['uniform','gradient_based'],
    "subsample":[.01,.02]
}

In [6]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [7]:

## Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
classifier=xgboost.XGBClassifier()
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [8]:
from datetime import datetime
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train,y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  25 | elapsed:  8.3min remaining: 33.3min
[Parallel(n_jobs=-1)]: Done  14 out of  25 | elapsed:  8.7min remaining:  6.8min
[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed: 12.6min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 12.8min finished



 Time taken: 0 hours 13 minutes and 54.2 seconds.


In [9]:
random_search.best_estimator_


XGBClassifier(alpha=1, base_score=0.5, booster='dart', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, eta=0.2, gamma=0.3,
              gpu_id=-1, grow_policy='depthwise', importance_type='gain',
              interaction_constraints='', learning_rate=0.3, max_delta_step=0,
              max_depth=8, min_child_weight=29, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=1, reg_lambda=1,
              sampling_method='uniform', scale_pos_weight=1, subsample=0.01,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
random_search.best_params_

{'tree_method': 'approx',
 'subsample': 0.01,
 'sampling_method': 'uniform',
 'reg_lambda': 1,
 'n_estimators': 100,
 'min_child_weight': 11,
 'max_depth': 35,
 'learning_rate': 0.3,
 'grow_policy': 'depthwise',
 'gamma': 0.2,
 'eta': 0.4,
 'colsample_bytree': 0.5,
 'booster': 'gbtree',
 'alpha': 3}